In [ ]:
import tensorflow as tf
tf.__version__

'2.14.0'

In [ ]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_dir = 'dataset'
if not os.path.exists(dataset_dir):
    os.mkdir(dataset_dir)

In [ ]:
target_file ='/content/drive/MyDrive/Tomato/DATA.zip'
with zipfile.ZipFile(target_file, 'r') as extracting:
    extracting.extractall(dataset_dir)

In [ ]:
raw_dir = os.path.join(dataset_dir, 'DATA', 'BELUM SIAP')

ripe_dir = os.path.join(dataset_dir, 'DATA', 'SIAP PANEN')

In [ ]:
print(len(os.listdir(raw_dir)))
print(len(os.listdir(ripe_dir)))

900
900


In [ ]:
print(os.listdir(raw_dir)[0])

belum_siap__0_9026.jpeg


In [ ]:
# Define root directory
root_dir = '/content/dataset/DATA/trainval/'

# Empty directory to prevent FileExistsError is the function is run several times
if os.path.exists(root_dir):
  shutil.rmtree(root_dir)

# Make function for crate train val dir
def create_train_val_dirs(root_path):

  # train dir and val dir
  train_dir = os.path.join(root_path,'training')
  val_dir = os.path.join(root_path, 'validation')

  os.makedirs(train_dir)
  os.makedirs(val_dir)
  # Create subdirectories for cats and dogs in the training set
  tsiap = os.path.join(train_dir, 'BELUM SIAP')
  tblmsiap = os.path.join(train_dir, 'SIAP PANEN')

  os.makedirs(tsiap)
  os.makedirs(tblmsiap)

  # Create subdirectories for cats and dogs in the validation set
  vsiap= os.path.join(val_dir, 'BELUM SIAP')
  vblmsiap = os.path.join(val_dir, 'SIAP PANEN')



  os.makedirs(vsiap)
  os.makedirs(vblmsiap)


try:
  create_train_val_dirs(root_path=root_dir)
except FileExistsError:
  print("You should not be seeing this since the upper directory is removed beforehand")

In [ ]:
for rootdir, dirs, files in os.walk(root_dir):
    for subdir in dirs:
        print(os.path.join(rootdir, subdir))

/content/dataset/DATA/trainval/training
/content/dataset/DATA/trainval/validation
/content/dataset/DATA/trainval/training/SIAP PANEN
/content/dataset/DATA/trainval/training/BELUM SIAP
/content/dataset/DATA/trainval/validation/SIAP PANEN
/content/dataset/DATA/trainval/validation/BELUM SIAP


In [ ]:
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):
  all_files = []
  file_list = os.listdir(SOURCE_DIR)

  for file_name in file_list:
    file_path = os.path.join(SOURCE_DIR, file_name)

    file_size = os.path.getsize(file_path)
    if file_size == 0 :
      print('{} is zero length, so ignoring'.format(file_name))
    else:
      all_files.append(file_name)

  shuffled_files = random.sample(all_files, len(all_files))
  split_index = int(len(all_files) * SPLIT_SIZE)
  training_files = shuffled_files[:split_index]
  validation_files = shuffled_files[split_index:]

  for file in training_files:
    copyfile(os.path.join(SOURCE_DIR, file), os.path.join(TRAINING_DIR, file))

  for file in validation_files:
    copyfile(os.path.join(SOURCE_DIR, file), os.path.join(VALIDATION_DIR, file))

In [ ]:
SIAP = "/content/dataset/DATA/SIAP PANEN"
BELUM_SIAP= "/content/dataset/DATA/BELUM SIAP/"

TRAINING_DIR = "/content/dataset/DATA/trainval/training/"
VALIDATION_DIR = "/content/dataset/DATA/trainval/validation/"

TRAINING_SIAP_DIR = os.path.join(TRAINING_DIR, "SIAP PANEN")
TRAINING_BLMSIAP_DIR = os.path.join(TRAINING_DIR, "BELUM SIAP")

VALIDATION_SIAP_DIR = os.path.join(VALIDATION_DIR, "SIAP PANEN")
VALIDATION_BLMSIAP_DIR = os.path.join(VALIDATION_DIR, "BELUM SIAP")

# Define proportion of images used for training
split_size = .9

# Run the function
# NOTE: Messages about zero length images should be printed out
split_data(SIAP, TRAINING_SIAP_DIR, VALIDATION_SIAP_DIR, split_size)
split_data(BELUM_SIAP, TRAINING_BLMSIAP_DIR, VALIDATION_BLMSIAP_DIR, split_size)


In [ ]:
print(f"There are {len(os.listdir(TRAINING_SIAP_DIR))} images of  for training")
print(f"There are {len(os.listdir(TRAINING_BLMSIAP_DIR))} images of for training")


print(f"There are {len(os.listdir(VALIDATION_SIAP_DIR))} images of  validation")
print(f"There are {len(os.listdir(VALIDATION_BLMSIAP_DIR))} images of for validation")


There are 810 images of  for training
There are 810 images of for training
There are 90 images of  validation
There are 90 images of for validation


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(150,150),
	class_mode='binary',
  batch_size=20
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(150,150),
	class_mode='binary',
  batch_size=20
)

Found 1620 images belonging to 2 classes.
Found 180 images belonging to 2 classes.


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(metrics = ['accuracy'], loss = 'binary_crossentropy',
    optimizer = optimizer)


In [ ]:
# Constant for epochs
EPOCHS = 10


# Train the model
history = model.fit(
      train_generator,
      epochs=EPOCHS,
      validation_data=validation_generator,
      verbose=1)

Epoch 1/20
81/81 [==============================] - 275s 3s/step - loss: 0.4812 - accuracy: 0.7765 - val_loss: 0.2332 - val_accuracy: 0.9111
Epoch 2/20
81/81 [==============================] - 274s 3s/step - loss: 0.2807 - accuracy: 0.9006 - val_loss: 0.2410 - val_accuracy: 0.9167
Epoch 3/20
81/81 [==============================] - 275s 3s/step - loss: 0.2503 - accuracy: 0.9031 - val_loss: 0.1645 - val_accuracy: 0.9167
Epoch 4/20
81/81 [==============================] - 275s 3s/step - loss: 0.2034 - accuracy: 0.9191 - val_loss: 0.1533 - val_accuracy: 0.9444
Epoch 5/20
81/81 [==============================] - 276s 3s/step - loss: 0.1999 - accuracy: 0.9136 - val_loss: 0.1266 - val_accuracy: 0.9444
Epoch 6/20
81/81 [==============================] - 275s 3s/step - loss: 0.1664 - accuracy: 0.9321 - val_loss: 0.1984 - val_accuracy: 0.9167
Epoch 7/20
81/81 [==============================] - 275s 3s/step - loss: 0.1706 - accuracy: 0.9265 - val_loss: 0.1478 - val_accuracy: 0.9556
Epoch 8/20
81

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

# Plot the results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_acc = range(len(acc))

plt.plot(epochs_acc, acc, 'r', label='Training accuracy')
plt.plot(epochs_acc, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

epochs_loss = range(len(loss))

plt.plot(epochs_loss, loss, 'r', label='Training loss')
plt.plot(epochs_loss, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
import numpy as np

from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded=files.upload()

for fn in uploaded.keys():

  # predicting images
  path='/content/' + fn
  img=load_img(path, target_size=(150, 150))

  x=img_to_array(img)
  x /= 255
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])

  classes = model.predict(images, batch_size=10)

  print(classes[0])

  if classes[0]>0.5:
    print(fn + " SIAP")
  else:
    print(fn + " BELUM SIAP")

In [ ]:
#CALLBACK
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('loss') < 0.15):

      print("\nLoss is lower than 0.4 so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()